In [3]:
#using DifferentialEquations
using Plots
using Statistics
using FFTW
FFTW.set_provider!("mkl")
#import Pkg; Pkg.add("FileIO")
using FileIO
using JLD2
using Base.Threads
using Distributions
using LaTeXStrings
using Printf
using LsqFit

In [10]:
include("../aux_functions.jl")

temperature_fit (generic function with 1 method)

In [17]:
J = 200
N = 8_000
M_g = 11
L = 200
dx = L/J
order = 5
par_grid = (N, L, J, dx, order)
v = rand(2N+J,M_g);
κ = 2π/L

t_i = 0.0
t_f = 2.0
M = 2001
dt = (t_f - t_i)/(M_g - 1)
par_evolv = (t_i, t_f, M, M_g, dt)

θ = 0.001
nm = 2
par_f = (θ, nm, κ)

(0.001, 2, 0.031415926535897934)

In [13]:
n = zeros(J)
ϕ = zeros(J)
Tn = zeros(J, nthreads())
par_density = (par_grid, Tn)
E_f = zeros(J)
get_density!(v[:,end], n, par_grid)
get_ϕ!(ϕ, n .- 1.0, κ)
get_E_from_ϕ!(ϕ,E_f,dx)
E_f_t = zeros(J)
get_density_threads!(v[:,end], n, par_density)
get_ϕ!(ϕ, n .- 1.0, κ)
get_E_from_ϕ!(ϕ,E_f_t,dx)
(E_f - E_f_t)'*(E_f - E_f_t)

9.943231989738073e-24

In [18]:
@time Energy_K, Energy_E, EField_T, p_T, Q_T, S_T, T = get_averages(v,par_grid,par_evolv, par_f);


  0.374637 seconds (575.94 k allocations: 35.810 MiB, 65.83% compilation time)


In [19]:
@time Energy_K_t, Energy_E_t, EField_T_t, p_T_t, Q_T_t, S_T_t, T_t = get_averages_threads(v,par_grid,par_evolv, par_f);

  0.260180 seconds (310.03 k allocations: 21.270 MiB, 78.89% compilation time)


Testing v2p and p2v



In [11]:
p = rand(10)
m=2
v = p2v.(p;m=m)
p - v2p.(v;m=m)

10-element Vector{Float64}:
  0.0
 -5.551115123125783e-17
  0.0
 -5.551115123125783e-17
  0.0
 -1.1102230246251565e-16
  0.0
  0.0
  2.220446049250313e-16
 -1.1102230246251565e-16

In [12]:
v = p2v.(p)
p - v2p.(v)

10-element Vector{Float64}:
  0.0
  0.0
 -1.1102230246251565e-16
 -5.551115123125783e-17
  0.0
  0.0
  0.0
  0.0
  0.0
 -2.220446049250313e-16